In [23]:
import numpy as np
import torch
import os
import subprocess
from pathlib import Path

In [26]:
def generate_data(batch_size, path="./stls", start=0, scale=1):
    cmd = [
            "./scripts/generate_limbs.sh",
            "--num_limbs",
            f"{batch_size}",
            "--path",
            str(path),
            "--start",
            f"{start}",
            "--save_mesh",
            "0",
            "--scale",
            f"{scale}"
        ]

    if os.name == "nt":  # Windows
        cmd = ["wsl", "-e"] + cmd

    subprocess.run(cmd, check=False, stderr=subprocess.PIPE, stdout=subprocess.PIPE)

    while True:
        try:
            components = np.load(f"{path}/components_{start:08d}.npy")
        except FileNotFoundError:
            continue
        else:
            return torch.tensor(components)


unscaled = generate_data(1024, start=0, scale=0)
scaled = generate_data(1024, start=1024, scale=1)

scaled.shape, unscaled.shape
# Calculate summary statistics for each component in unscaled and scaled tensors
def summarize_tensor(tensor, name):
    stats = {
        'mean': tensor.mean(dim=0),
        'std': tensor.std(dim=0),
        'min': tensor.min(dim=0).values,
        'max': tensor.max(dim=0).values,
    }
    print(f"Summary statistics for {name}:")
    for i in range(tensor.shape[1]):
        print(f"Component {i}: mean={stats['mean'][i]:.4f}, std={stats['std'][i]:.4f}, min={stats['min'][i]:.4f}, max={stats['max'][i]:.4f}")
    print()

summarize_tensor(unscaled, "unscaled")
summarize_tensor(scaled, "scaled")

Summary statistics for unscaled:
Component 0: mean=6.1444, std=14.2853, min=-19.9567, max=33.8261
Component 1: mean=2.6163, std=4.9341, min=-6.3180, max=12.6947
Component 2: mean=0.5011, std=3.1790, min=-6.7520, max=8.9552
Component 3: mean=0.5367, std=2.7262, min=-5.9810, max=7.9328
Component 4: mean=0.9793, std=2.4555, min=-4.8135, max=6.8879
Component 5: mean=-0.2139, std=1.7827, min=-4.7284, max=4.3558
Component 6: mean=0.2772, std=1.7266, min=-3.5803, max=4.5307
Component 7: mean=-0.0566, std=1.2382, min=-2.9769, max=2.7980
Component 8: mean=0.0421, std=0.8715, min=-2.2610, max=2.1669
Component 9: mean=0.2130, std=1.0338, min=-2.0038, max=2.5253
Component 10: mean=1.0000, std=0.0000, min=1.0000, max=1.0000

Summary statistics for scaled:
Component 0: mean=6.1444, std=14.2853, min=-19.9567, max=33.8261
Component 1: mean=2.6163, std=4.9341, min=-6.3180, max=12.6947
Component 2: mean=0.5011, std=3.1790, min=-6.7520, max=8.9552
Component 3: mean=0.5367, std=2.7262, min=-5.9810, max=7.

In [31]:
scaled_component_transforms = torch.vstack([torch.mean(scaled, dim=0, keepdim=True), torch.std(scaled, dim=0, keepdim=True)])
unscaled_component_transforms = torch.vstack([torch.mean(unscaled, dim=0, keepdim=True), torch.std(unscaled, dim=0, keepdim=True)])

torch.save(scaled_component_transforms, "./data_components/scaled_component_transforms.pt")
torch.save(unscaled_component_transforms, "./data_components/unscaled_component_transforms.pt")

scaled_component_transforms.shape

torch.Size([2, 11])